In [1]:
import os
import keras 
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Dropout, Dense
from keras.models import Sequential
from keras import applications
from keras.activations import relu
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model

from keras import callbacks

Using TensorFlow backend.


In [2]:
train_data_dir= 'data/train'
test_data_dir = 'data/test'
batch_size = 10
no_classes = 2
epochs = 10

top_model_weights_path = 'top_model.h5'
vgg_weight = ''
img_height = 299
img_width = 299

nb_train_samples = 1900
nb_validation_samples = 100

In [3]:
base_model = applications.VGG16(include_top=False,weights='imagenet',input_shape=[299,299,3])

In [4]:
# model.summary

In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.6))
top_model.add(Dense(1,activation='sigmoid'))


In [7]:
top_model.load_weights(top_model_weights_path)
# model.add(top_model)
model = Model(inputs= base_model.inputs, outputs= top_model(base_model.outputs))


In [8]:
## let top 25 layers of VGG net, set to non trainable
for layer in model.layers[:15]:
    layer.trainable=False

In [9]:
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
             metrics=['accuracy'],loss='binary_crossentropy')

In [10]:
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2, 
                                   shear_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=[img_height,img_width],
                                                    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(test_data_dir,target_size=[img_height,img_width],
                                                    batch_size=batch_size, class_mode='binary')


Found 1900 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [11]:
# fine-tune the model
# model.fit_generator(
#     train_generator,
#     samples_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=nb_validation_samples // batch_size,
#     nb_val_samples=nb_validation_samples)

# add callback for tensorboard
tb_callbacks =callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                    write_graph=True, write_images=True)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
190/190 [==============================] - 365s - loss: 7.8985 - acc: 0.5058 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/10
190/190 [==============================] - 357s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/10
190/190 [==============================] - 355s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
  4/190 [..............................] - ETA: 327s - loss: 6.8502 - acc: 0.5750

KeyboardInterrupt: 